In [103]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import  AdaBoostClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn import model_selection


url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/molecular-biology/promoter-gene-sequences/promoters.data'
names = ['Class', 'id', 'Sequence']
data = pd.read_csv(url, names = names)

In [105]:
classes = data.loc[:, 'Class']
print(classes[:5])

0    +
1    +
2    +
3    +
4    +
Name: Class, dtype: object


In [ ]:
sequences = list(data.loc[:, 'Sequence'])
dataset = {}

for i, seq in enumerate(sequences):
    
    nucleotides = list(seq)
    nucleotides = [x for x in nucleotides if x != '\t']
    
    nucleotides.append(classes[i])
    
    dataset[i] = nucleotides
# print(dataset[0])

In [ ]:
# turn dataset into pandas DataFrame
dframe = pd.DataFrame(dataset)
#print(dframe)
df = dframe.transpose()
df.rename(columns = {57: 'Class'}, inplace = True)


In [111]:
df.describe()
numerical_df = pd.get_dummies(df)
# drop one class (-) 
df = numerical_df.drop(columns=['Class_-'])
df.rename(columns = {'Class_+': 'Class'}, inplace = True)
print(df)

     0_a  0_c  0_g  0_t  1_a  1_c  1_g  1_t  2_a  2_c  ...    54_t  55_a  \
0      0    0    0    1    1    0    0    0    0    1  ...       0     0   
1      0    0    0    1    0    0    1    0    0    1  ...       0     1   
2      0    0    1    0    0    0    0    1    1    0  ...       0     0   
3      1    0    0    0    1    0    0    0    0    0  ...       0     0   
4      0    0    0    1    0    1    0    0    0    0  ...       1     1   
5      1    0    0    0    0    0    1    0    0    0  ...       0     0   
6      0    1    0    0    1    0    0    0    0    0  ...       0     1   
7      0    0    0    1    0    0    0    1    0    0  ...       0     0   
8      0    1    0    0    0    0    1    0    1    0  ...       1     0   
9      0    0    0    1    0    0    0    1    0    0  ...       0     0   
10     0    0    1    0    0    1    0    0    1    0  ...       0     0   
11     0    1    0    0    0    1    0    0    0    0  ...       0     1   
12     0    

In [112]:

# Create X and Y datasets
X = np.array(df.drop(['Class'], 1))
y = np.array(df['Class'])

seed = 1

# split data into training and testing datasets
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=seed)

In [142]:
scoring = 'accuracy'

names = ["Neural Net","SVM Linear", "AdaBoost"]
classifiers = [
     MLPClassifier(alpha=1,batch_size=30),
     SVC(kernel = 'linear',), 
     AdaBoostClassifier(),
]

models = zip(names, classifiers)


results = []
names = []

for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state = seed)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
    print('--------------------------------------------------')
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(name)
    print(accuracy_score(y_test, predictions))
    print(classification_report(y_test, predictions))
    print('--------------------------------------------------')

Neural Net: 0.900000 (0.075000)
--------------------------------------------------
Neural Net
0.8888888888888888
             precision    recall  f1-score   support

          0       1.00      0.82      0.90        17
          1       0.77      1.00      0.87        10

avg / total       0.91      0.89      0.89        27

--------------------------------------------------
SVM Linear: 0.850000 (0.108972)
--------------------------------------------------
SVM Linear
0.9629629629629629
             precision    recall  f1-score   support

          0       1.00      0.94      0.97        17
          1       0.91      1.00      0.95        10

avg / total       0.97      0.96      0.96        27

--------------------------------------------------
AdaBoost: 0.925000 (0.114564)
--------------------------------------------------
AdaBoost
0.8518518518518519
             precision    recall  f1-score   support

          0       1.00      0.76      0.87        17
          1       0.71    